<a href="https://colab.research.google.com/github/Hisernberg/ai-agent-projects/blob/main/webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00


In [2]:
import requests

# Groq API endpoint and API key
groq_api_url = 'https://api.groq.com/v1/ollama'  # Update this based on your actual endpoint
api_key = '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'  # Replace with your actual API key

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}


In [3]:
from bs4 import BeautifulSoup
import requests

def fetch_website_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove unwanted elements like scripts and styles
    for script in soup(["script", "style"]):
        script.decompose()

    # Extract and combine text from paragraphs
    text_content = ' '.join(p.get_text() for p in soup.find_all('p'))
    return text_content


In [4]:
def summarize_with_groq(url):
    url = fix_url(url)  # Ensure URL is well-formed
    website_content = fetch_website_content(url)

    if not website_content.strip():
        return "No content found on this website."

    prompt = f"Please summarize the following website content:\n\n{website_content}"

    payload = {
        'model': 'llama2',
        'prompt': prompt,
    }

    response = requests.post(groq_api_url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        summary = result.get('text', 'No summary available')
    else:
        summary = f"An error occurred: {response.status_code} - {response.text}"

    return summary


In [5]:
def fix_url(url):
    if not url.startswith("http://") and not url.startswith("https://"):
        url = "https://" + url
    return url


In [6]:
def on_button_click(b):
    url = url_input.value.strip()
    if url:
        with output:
            output.clear_output()
            summary = summarize_with_groq(url)
            display(Markdown(summary))


In [7]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Create a text box for URL input
url_input = widgets.Text(
    value='',
    placeholder='Enter the URL of the website',
    description='Website URL:',
    disabled=False
)

# Button to trigger summarization
summarize_button = widgets.Button(description="Summarize")

# Output widget to display the result
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    url = url_input.value
    if url:
        with output:
            summary = summarize_with_groq(url)
            display(Markdown(summary))  # Display the summary in Markdown format

# Attach the function to the button
summarize_button.on_click(on_button_click)

# Display the widgets in the notebook
display(url_input, summarize_button, output)


Text(value='', description='Website URL:', placeholder='Enter the URL of the website')

Button(description='Summarize', style=ButtonStyle())

Output()

In [9]:
import requests
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, Markdown

# --- Replace these with your real Groq API details ---
groq_api_url = 'https://api.groq.com/openai/v1/chat/completions'
api_key = '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'

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}

# --- Utility: Fix URLs if missing http/https ---
def fix_url(url):
    if not url.startswith("http://") and not url.startswith("https://"):
        url = "https://" + url
    return url

# --- Scrape website content ---
def fetch_website_content(url):
    url = fix_url(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove scripts and styles
    for tag in soup(["script", "style"]):
        tag.decompose()

    text = ' '.join(p.get_text() for p in soup.find_all('p'))
    return text.strip()

# --- Summarize using Groq (Ollama) ---
def summarize_with_groq(url):
    url = fix_url(url)
    content = fetch_website_content(url)

    if not content:
        return "No readable content found on the website."

    prompt = f"Summarize the following website content:\n\n{content[:4000]}"  # Truncate to avoid huge prompts

    payload = {
        'model': 'llama2',  # Or other model you prefer
        'prompt': prompt,
    }

    response = requests.post(groq_api_url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result.get('text', 'No summary returned.')
    else:
        return f"Error {response.status_code}: {response.text}"


In [18]:
# Predefined URLs
url_options = {
    "CNN": "https://cnn.com",
    "Anthropic": "https://anthropic.com"
}

dropdown = widgets.Dropdown(
    options=url_options,
    description='Choose Site:',
)

summarize_button = widgets.Button(description="Summarize Site")
output = widgets.Output()

def on_button_click(b):
    output.clear_output()
    selected_url = dropdown.value
    with output:
        summary = summarize_with_groq(selected_url)
        display(Markdown(f"### Summary for [{selected_url}]({selected_url}):\n\n{summary}"))

summarize_button.on_click(on_button_click)

display(dropdown, summarize_button, output)


Dropdown(description='Choose Site:', options={'CNN': 'https://cnn.com', 'Anthropic': 'https://anthropic.com'},…

Button(description='Summarize Site', style=ButtonStyle())

Output()

In [12]:
def summarize_with_groq(url):
    url = fix_url(url)
    content = fetch_website_content(url)

    if not content:
        return "No readable content found on the website."

    system_prompt = "You are an assistant that summarizes website content in markdown. Ignore navigation links or irrelevant UI text."
    user_prompt = f"Summarize the following website in markdown:\n\n{content[:4000]}"

    payload = {
        "model": "llama3-8b-8192",  # Or "mixtral-8x7b-32768"
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.7
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        return f"Error {response.status_code}: {response.text}"


In [14]:
def fetch_website_content(url):
    url = fix_url(url)
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove scripts and styles
    for tag in soup(["script", "style", "noscript", "header", "footer", "nav", "aside"]):
        tag.decompose()

    # Try to focus on article content
    article_text = []
    for tag in soup.find_all(["h1", "h2", "p", "article"]):
        text = tag.get_text(strip=True)
        if text and len(text.split()) > 5:
            article_text.append(text)

    return "\n".join(article_text[:100])  # Truncate to first 100 paragraphs/sentences


In [17]:
!pip install newspaper3k
!pip install lxml_html_clean

from newspaper import Article

from newspaper import Article

def fetch_website_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text



In [21]:
# 🛠️ Install dependencies
!pip install newspaper3k lxml_html_clean ipywidgets --quiet
import requests
import ipywidgets as widgets
from IPython.display import display, Markdown
from newspaper import Article

# 🔐 Groq API setup
GROQ_API_KEY = "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"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

# 📄 Extract article content using newspaper3k
def fetch_website_content(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return f"Error fetching content: {str(e)}"

# 🧠 Build message payload
def build_prompt(text, title="Website"):
    return [
        {
            "role": "system",
            "content": "You are a helpful assistant that summarizes websites in markdown. Focus on the main content. Ignore menus, ads, and footers.",
        },
        {
            "role": "user",
            "content": f"You are viewing a website titled {title}. Here's the content:\n\n{text}\n\nPlease summarize the website in markdown.",
        },
    ]

# 🚀 Request to Groq (LLaMA3, Mixtral, etc.)
def summarize_with_groq(url):
    content = fetch_website_content(url)
    if not content or "Error" in content:
        return "Failed to extract website content."

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-70b-8192",  # or "mixtral-8x7b-32768"
        "messages": build_prompt(content, title=url),
        "temperature": 0.7
    }

    try:
        response = requests.post(GROQ_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Error from Groq API: {str(e)}"

# 🔘 UI: Dropdown for site selection and summarize button
url_options = {
    "CNN": "https://cnn.com",
    "Anthropic": "https://anthropic.com"
}

dropdown = widgets.Dropdown(
    options=url_options,
    description='Choose Site:',
)

summarize_button = widgets.Button(description="Summarize Site")
output = widgets.Output()

def on_button_click(b):
    output.clear_output()
    selected_url = dropdown.value
    with output:
        summary = summarize_with_groq(selected_url)
        display(Markdown(f"### Summary for [{selected_url}]({selected_url}):\n\n{summary}"))

summarize_button.on_click(on_button_click)

display(dropdown, summarize_button, output)


Dropdown(description='Choose Site:', options={'CNN': 'https://cnn.com', 'Anthropic': 'https://anthropic.com'},…

Button(description='Summarize Site', style=ButtonStyle())

Output()